In [1]:
import datetime as dt
import math

import matplotlib.pyplot as plt
import mplfinance as mpf
import numpy as np
import pandas as pd
import pandas_market_calendars as mcal
import plotly.graph_objects as go
import polars as pl
from dash import Dash, dcc, html
from plotly.subplots import make_subplots

nse = mcal.get_calendar("NSE")

pd.set_option("display.max_rows", 25_000)
pd.set_option("display.max_columns", 500)
pl.Config.set_tbl_cols(500)
pl.Config.set_tbl_rows(10_000)

pd.options.display.float_format = "{:.4f}".format

import sys

sys.path.append("..")
# from tooling.enums import AssetClass, Index, Spot, StrikeSpread
# from tooling.fetch import fetch_option_data, fetch_spot_data
# from tooling.filter import find_atm, option_tool

from fetching_from_local_db.enums import AssetClass, Index, StrikeSpread
from fetching_from_local_db.fetch_from_db import (
    _fetch_batch,
    fetch_data,
    fetch_spot_data,
)

In [2]:
# bnf_pandas = pd.read_csv("../data/bnf_min.csv")
bnf_pandas = pd.read_csv("../data/ng_1hr.csv")
# bnf_pandas = pd.read_csv("../data/fin_min.csv")
# bnf_pandas = pd.read_csv("../data/midcp_min.csv")
# bnf_pandas = pd.read_csv("../data/sensex_min.csv")
# bnf_pandas = pd.read_csv("../data/bankex_min.csv")

In [3]:
# If Stocks Data ...
bnf_pandas["time"] = pd.to_datetime(bnf_pandas["time"])
bnf_pandas["time"] = bnf_pandas["time"].dt.tz_localize(None)
bnf_pandas = bnf_pandas[bnf_pandas["time"].dt.year >= 2017]
bnf_pandas['datetime'] = bnf_pandas['time']
bnf_pandas.drop(columns=["time"], inplace=True)
# bnf_pandas

In [4]:
bnf = pl.DataFrame(bnf_pandas)
print(type(bnf))
# bnf

<class 'polars.dataframe.frame.DataFrame'>


In [5]:
bnf = bnf.with_columns([pl.col("datetime").alias("index")]).drop("datetime")
bnf = bnf.with_columns(pl.col("index").alias("datetime"))

In [6]:
bnf = bnf.rename({"open": "o", "high": "h", "low": "l", "close": "c"})

In [7]:
bnf.tail()

o,h,l,c,index,datetime
f64,f64,f64,f64,datetime[ns],datetime[ns]
324.2,329.8,323.1,327.0,2024-12-24 20:00:00,2024-12-24 20:00:00
327.0,332.5,327.0,331.4,2024-12-24 21:00:00,2024-12-24 21:00:00
331.3,332.8,326.6,332.4,2024-12-24 22:00:00,2024-12-24 22:00:00
332.2,336.9,329.6,334.1,2024-12-24 23:00:00,2024-12-24 23:00:00
328.9,328.9,327.3,327.7,2024-12-26 09:00:00,2024-12-26 09:00:00


In [24]:
def trade(df, x=0.02, y=0.01):
    # Ensure datetime column is in the correct format
    df['datetime'] = pd.to_datetime(df['datetime'])

    # Set datetime as the index for time-based operations
    df = df.set_index('datetime')

    # Create a date column for grouping
    df['date'] = df.index.date

    # Calculate daily range (high - low) for each day between 9 AM and 3 PM
    intraday_df = df.between_time("09:00", "15:00")
    daily_range = intraday_df.groupby('date').agg({'h': 'max', 'l': 'min'})
    daily_range['range'] = daily_range['h'] - daily_range['l']

    # Initialize tradebook
    tradebook = []

    # Iterate over days to simulate trades
    for date, data in df.groupby('date'):
        if len(data) < 2:
            continue  # Skip if insufficient data

        # Extract 4 PM open price
        open_price_4pm = data.between_time("16:00", "16:00")['o']
        if open_price_4pm.empty:
            continue

        open_price_4pm = open_price_4pm.iloc[0]
        day_high = daily_range.loc[date, 'h']
        day_low = daily_range.loc[date, 'l']
        day_range = daily_range.loc[date, 'range']
        long_entry_price = open_price_4pm + x * day_range
        short_entry_price = open_price_4pm - x * day_range

        # Find trades after 4 PM
        trades = data[data.index >= pd.to_datetime(f"{date} 16:00:00")]

        # Get the last candle of the day
        last_candle = trades.iloc[-1] if not trades.empty else None

        # Track active trades and daily trade limits
        active_long = False
        active_short = False
        long_trade_done = False
        short_trade_done = False

        take_exit_long = False
        take_exit_short = False

        for idx, row in trades.iterrows():
            # Check for long entry
            if not active_long and not long_trade_done and row['h'] >= long_entry_price:
                active_long = True
                long_trade_done = True  # Mark long trade as done for the day
                sl = long_entry_price - y * day_range
                target = long_entry_price + 2 * y * day_range
                entry_time = idx

            if active_long:
                # Determine exit
                if row['l'] <= sl:  # Stop-loss hit
                    exit_price = sl
                    exit_time = idx
                    take_exit_long = True
                    exit_remark_long = 'SL'
                elif row['h'] >= target:  # Target hit
                    exit_price = target
                    exit_time = idx
                    take_exit_long = True
                    exit_remark_long = 'Target'
                elif last_candle is not None:  # End of the day exit
                    exit_price = last_candle['c']
                    exit_time = last_candle.name
                    take_exit_long = True
                    exit_remark_long = 'EOD'
                else:
                    continue

                if take_exit_long:
                    points_captured = exit_price - long_entry_price

                    tradebook.append({
                        'entry_time': entry_time,
                        'exit_time': exit_time,
                        'entry_price': long_entry_price,
                        'exit_price': exit_price,
                        'points_captured': points_captured,
                        'remark': exit_remark_long,
                        'trade_type': 'Long',
                        'day_high_till_3pm': day_high,
                        'day_low_till_3pm': day_low,
                        'day_range': day_range,
                    })

                    active_long = False  # Reset active trade after exit
                    take_exit_long = False

            # Check for short entry
            if not active_short and not short_trade_done and row['l'] <= short_entry_price:
                active_short = True
                short_trade_done = True  # Mark short trade as done for the day
                sl = short_entry_price + y * day_range
                target = short_entry_price - 2 * y * day_range
                entry_time = idx

            if active_short:
                # Determine exit
                if row['h'] >= sl:  # Stop-loss hit
                    exit_price = sl
                    exit_time = idx
                    take_exit_short = True
                    exit_remark_short = 'SL'
                elif row['l'] <= target:  # Target hit
                    exit_price = target
                    exit_time = idx
                    take_exit_short = True
                    exit_remark_short = 'Target'
                elif last_candle is not None:  # End of the day exit
                    exit_price = last_candle['c']
                    exit_time = last_candle.name
                    take_exit_short = True
                    exit_remark_short = 'EOD'
                else:
                    continue

                if take_exit_short:
                    points_captured = short_entry_price - exit_price

                    tradebook.append({
                        'entry_time': entry_time,
                        'exit_time': exit_time,
                        'entry_price': short_entry_price,
                        'exit_price': exit_price,
                        'points_captured': points_captured,
                        'remark': exit_remark_short,
                        'trade_type': 'Short',
                        'day_high_till_3pm': day_high,
                        'day_low_till_3pm': day_low,
                        'day_range': day_range,
                    })

                    active_short = False  # Reset active trade after exit
                    take_exit_short = False

    # Create DataFrame for tradebook
    tradebook_df = pd.DataFrame(tradebook)
    return tradebook_df


In [30]:
data = bnf
df = pd.DataFrame(data)
df.columns = ['o', 'h', 'l', 'c', 'index', 'datetime']
tradebook = trade(df, x=0.02, y=0.02)
# print(tradebook)

In [31]:
tradebook

,entry_time,exit_time,entry_price,exit_price,points_captured,remark,trade_type,day_high_till_3pm,day_low_till_3pm,day_range
0,2020-12-28 16:00:00,2020-12-28 16:00:00,168.6260,168.9000,-0.2740,SL,Short,180.0000,166.3000,13.7000
1,2020-12-28 22:00:00,2020-12-28 22:00:00,169.1740,168.9000,-0.2740,SL,Long,180.0000,166.3000,13.7000
2,2020-12-29 16:00:00,2020-12-29 16:00:00,172.4760,172.4000,-0.0760,SL,Long,173.9000,170.1000,3.8000
3,2020-12-30 16:00:00,2020-12-30 16:00:00,179.3540,179.3000,-0.0540,SL,Long,180.0000,177.3000,2.7000
4,2020-12-30 16:00:00,2020-12-30 16:00:00,179.2460,179.3000,-0.0540,SL,Short,180.0000,177.3000,2.7000
5,2020-12-31 16:00:00,2020-12-31 16:00:00,183.6200,183.5000,-0.1200,SL,Long,184.3000,178.3000,6.0000
6,2020-12-31 17:00:00,2020-12-31 17:00:00,183.3800,183.5000,-0.1200,SL,Short,184.3000,178.3000,6.0000
7,2021-01-01 16:00:00,2021-01-01 16:00:00,185.7940,185.9000,-0.1060,SL,Short,188.0000,182.7000,5.3000
8,2021-01-04 16:00:00,2021-01-04 16:00:00,191.8900,191.8000,-0.0900,SL,Long,193.6000,189.1000,4.5000
9,2021-01-04 16:00:00,2021-01-04 16:00:00,191.7100,191.8000,-0.0900,SL,Short,193.6000,189.1000,4.5000


In [32]:
tradebook['points_captured'].sum()

-288.31399975600004

In [34]:
tradebook.tail(30)

,entry_time,exit_time,entry_price,exit_price,points_captured,remark,trade_type,day_high_till_3pm,day_low_till_3pm,day_range
1905,2024-12-04 16:00:00,2024-12-04 16:00:00,256.8600,256.8000,-0.0600,SL,Long,259.6000,256.6000,3.0000
1906,2024-12-04 16:00:00,2024-12-04 16:00:00,256.7400,256.8000,-0.0600,SL,Short,259.6000,256.6000,3.0000
1907,2024-12-05 16:00:00,2024-12-05 16:00:00,262.8960,263.0000,-0.1040,SL,Short,264.1000,258.9000,5.2000
1908,2024-12-05 17:00:00,2024-12-05 17:00:00,263.1040,263.0000,-0.1040,SL,Long,264.1000,258.9000,5.2000
1909,2024-12-06 16:00:00,2024-12-06 16:00:00,257.0900,257.0000,-0.0900,SL,Long,261.4000,256.9000,4.5000
1910,2024-12-06 16:00:00,2024-12-06 16:00:00,256.9100,257.0000,-0.0900,SL,Short,261.4000,256.9000,4.5000
1911,2024-12-09 16:00:00,2024-12-09 16:00:00,275.0080,275.2000,-0.1920,SL,Short,275.4000,265.8000,9.6000
1912,2024-12-09 17:00:00,2024-12-09 17:00:00,275.3920,275.2000,-0.1920,SL,Long,275.4000,265.8000,9.6000
1913,2024-12-10 16:00:00,2024-12-10 16:00:00,267.8560,267.8000,-0.0560,SL,Long,270.5000,267.7000,2.8000
1914,2024-12-10 16:00:00,2024-12-10 16:00:00,267.7440,267.8000,-0.0560,SL,Short,270.5000,267.7000,2.8000
